# Working with Databases

In [1]:
#load watermark
%load_ext watermark
%watermark -a 'Gopala KR' -u -d -v -p watermark,numpy,pandas,matplotlib,nltk,sklearn,tensorflow,theano,mxnet,chainer,seaborn,keras,tflearn,bokeh,gensim

/srv/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
Using TensorFlow backend.
/srv/venv/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))


Gopala KR 
last updated: 2018-02-20 

CPython 3.6.3
IPython 6.2.1

watermark 1.6.0
numpy 1.14.0
pandas 0.22.0
matplotlib 2.1.2
nltk 3.2.5
sklearn 0.19.1
tensorflow 1.5.0
theano 1.0.1
mxnet 1.1.0
chainer 3.4.0
seaborn 0.8.1
keras 2.1.4
tflearn n
bokeh 0.12.14
gensim 3.3.0


/srv/venv/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


# sqlite3


In [2]:
!pip install sqlite3

  Could not find a version that satisfies the requirement sqlite3 (from versions: )
No matching distribution found for sqlite3


In [3]:
import sqlite3

with sqlite3.connect(":memory:") as con:
    c = con.cursor()
    c.execute('''CREATE TABLE sensors (date text, city text, code text, sensor_id real, temperature real)''')

    for table in c.execute("SELECT name FROM sqlite_master WHERE type = 'table'"):
        print("Table", table[0])

    c.execute("INSERT INTO sensors VALUES ('2016-11-05','Utrecht','Red',42,15.14)")
    c.execute("SELECT * FROM sensors")
    print(c.fetchone())
    con.execute("DROP TABLE sensors")

    print("# of tables", c.execute("SELECT COUNT(*) FROM sqlite_master WHERE type = 'table'").fetchone()[0])

    c.close()

Table sensors
('2016-11-05', 'Utrecht', 'Red', 42.0, 15.14)
# of tables 0


# pandas

In [4]:
import statsmodels.api as sm
from pandas.io.sql import read_sql
import sqlite3

with sqlite3.connect(":memory:") as con:
    c = con.cursor()

    data_loader = sm.datasets.sunspots.load_pandas()
    df = data_loader.data
    rows = [tuple(x) for x in df.values]

    con.execute("CREATE TABLE sunspots(year, sunactivity)")
    con.executemany("INSERT INTO sunspots(year, sunactivity) VALUES (?, ?)", rows)
    c.execute("SELECT COUNT(*) FROM sunspots")
    print(c.fetchone())
    print("Deleted", con.execute("DELETE FROM sunspots where sunactivity > 20").rowcount, "rows")

    print(read_sql("SELECT * FROM sunspots where year < 1732", con))
    con.execute("DROP TABLE sunspots")

    c.close()

/srv/venv/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


(309,)
Deleted 217 rows
      year  sunactivity
0   1700.0          5.0
1   1701.0         11.0
2   1702.0         16.0
3   1707.0         20.0
4   1708.0         10.0
5   1709.0          8.0
6   1710.0          3.0
7   1711.0          0.0
8   1712.0          0.0
9   1713.0          2.0
10  1714.0         11.0
11  1723.0         11.0


# SQLAlchemy

In [6]:
!pip install sqlalchemy

    100% |████████████████████████████████| 5.6MB 229kB/s eta 0:00:01
  Running setup.py bdist_wheel for sqlalchemy ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/f2/3c/ea/6dbd30a93900abc4fde20c65227fa98c4b714dfc997f44c495
Successfully built sqlalchemy


In [7]:
from sqlalchemy import Column, ForeignKey, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
from sqlalchemy import UniqueConstraint

Base = declarative_base()
class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    name = Column(String(14), nullable=False, unique=True)

    def __repr__(self):
        return "Id=%d name=%s" %(self.id, self.name)

class Sensor(Base):
    __tablename__ = 'sensor'
    id = Column(Integer, primary_key=True)
    last = Column(Integer)
    multiplier = Column(Float)
    station_id = Column(Integer, ForeignKey('station.id'))
    station = relationship(Station)

    def __repr__(self):
        return "Id=%d last=%d multiplier=%.1f station_id=%d" 
# %(self.id, self.last, self.multiplier, self.station_id)

if __name__ == "__main__":
    print("This script is used by code further down in this notebook.")

This script is used by code further down in this notebook.


In [8]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

#from alchemy_entities import Base, Sensor, Station

def populate(engine):
    Base.metadata.bind = engine

    DBSession = sessionmaker(bind=engine)
    session = DBSession()

    de_bilt = Station(name='De Bilt')
    session.add(de_bilt)
    session.add(Station(name='Utrecht'))
    session.commit()
    print("Station", de_bilt)

    temp_sensor = Sensor(last=20, multiplier=.1, station=de_bilt)
    session.add(temp_sensor)
    session.commit()
    print("Sensor", temp_sensor)

if __name__ == "__main__":
    print("This script is used by code further down in this notebook")

This script is used by code further down in this notebook


In [9]:
#from alchemy_entities import Base, Sensor, Station
#from populate_db import populate
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import os
from pandas.io.sql import read_sql


engine = create_engine('sqlite:///demo.db')
Base.metadata.create_all(engine)
populate(engine)
Base.metadata.bind = engine
DBSession = sessionmaker()
DBSession.bind = engine
session = DBSession()

station = session.query(Station).first()

print("Query 1", session.query(Station).all())
print("Query 2", session.query(Sensor).all())
print("Query 3", session.query(Sensor).filter(Sensor.station == station).one())
print(read_sql("SELECT * FROM station", engine.raw_connection()))

try:
    os.remove('demo.db')
    print("Deleted demo.db")
except OSError:
    pass

Station Id=1 name=De Bilt
Sensor Id=%d last=%d multiplier=%.1f station_id=%d
Query 1 [Id=1 name=De Bilt, Id=2 name=Utrecht]
Query 2 [Id=%d last=%d multiplier=%.1f station_id=%d]
Query 3 Id=%d last=%d multiplier=%.1f station_id=%d
   id     name
0   1  De Bilt
1   2  Utrecht
Deleted demo.db


# Pony ORM

In [11]:
!pip install dataset

    100% |████████████████████████████████| 1.0MB 1.1MB/s ta 0:00:01
    100% |████████████████████████████████| 573kB 1.8MB/s ta 0:00:01
  Running setup.py bdist_wheel for alembic ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/29/f4/c3/96037a3f2dcc2b8b59eff64746ea71bb5957f189c5a0877364
  Running setup.py bdist_wheel for Mako ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/33/bf/8f/036f36c35e0e3c63a4685e306bce6b00b6349fec5b0947586e
  Running setup.py bdist_wheel for python-editor ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/84/d6/b8/082dc3b5cd7763f17f5500a193b6b248102217cbaa3f0a24ca
Successfully built alembic Mako python-editor


# dataset

In [12]:
import dataset
from pandas.io.sql import read_sql
from pandas.io.sql import to_sql
import statsmodels.api as sm

db = dataset.connect('sqlite:///:memory:')
table = db["books"]
table.insert(dict(title="NumPy Beginner's Guide", author='Ivan Idris'))
table.insert(dict(title="NumPy Cookbook", author='Ivan Idris'))
table.insert(dict(title="Learning NumPy", author='Ivan Idris'))
print(read_sql('SELECT * FROM books', db.executable.raw_connection()) )

data_loader = sm.datasets.sunspots.load_pandas()
df = data_loader.data
#write_frame(df, "sunspots", db.executable.raw_connection()) 
df.to_sql("sunspots", db.executable.raw_connection()) 

table = db['sunspots']

for row in table.find(_limit=5):
   print(row)

print("Tables", db.tables)

AttributeError: 'Connection' object has no attribute 'raw_connection'

# mongodb

In [14]:
!pip install pymongo

    100% |████████████████████████████████| 378kB 2.3MB/s ta 0:00:01


In [15]:
from pymongo import MongoClient
import statsmodels.api as sm
import json
import pandas as pd

client = MongoClient()
db = client.test_database

data_loader = sm.datasets.sunspots.load_pandas()
df = data_loader.data
rows = json.loads(df.T.to_json()).values()
db.sunspots.insert_many(rows)

cursor = db['sunspots'].find({})
df =  pd.DataFrame(list(cursor))
print(df)

db.drop_collection('sunspots')

ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused

# Redis

In [16]:
!pip install redis

    100% |████████████████████████████████| 71kB 2.5MB/s ta 0:00:01


In [17]:
import redis
import statsmodels.api as sm
import pandas as pd

r = redis.StrictRedis()
data_loader = sm.datasets.sunspots.load_pandas()
df = data_loader.data
data = df.T.to_json()
r.set('sunspots', data)
blob = r.get('sunspots')
print(pd.read_json(blob))

ConnectionError: Error 111 connecting to localhost:6379. Connection refused.

# memcached

In [20]:
!pip install python-memcached

In [21]:
import memcache
import statsmodels.api as sm
import pandas as pd

client = memcache.Client([('127.0.0.1', 11211)])
data_loader = sm.datasets.sunspots.load_pandas()
df = data_loader.data
data = df.T.to_json()
client.set('sunspots', data, time=600)
print("Stored data to memcached, auto-expire after 600 seconds")
blob = client.get('sunspots')
print(pd.read_json(blob))

Stored data to memcached, auto-expire after 600 seconds


ValueError: Invalid file path or buffer object type: <class 'NoneType'>

# Cassandra

In [22]:
!pip install cassandra

  Could not find a version that satisfies the requirement cassandra (from versions: )
No matching distribution found for cassandra


In [23]:
from cassandra import ConsistencyLevel
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement
import statsmodels.api as sm

cluster = Cluster()
session = cluster.connect()
session.execute("CREATE KEYSPACE IF NOT EXISTS mykeyspace WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 };")
session.set_keyspace('mykeyspace')
session.execute("CREATE TABLE IF NOT EXISTS sunspots (year decimal PRIMARY KEY, sunactivity decimal);")

query = SimpleStatement(
    "INSERT INTO sunspots (year, sunactivity) VALUES (%s, %s)",
    consistency_level=ConsistencyLevel.QUORUM)

data_loader = sm.datasets.sunspots.load_pandas()
df = data_loader.data
rows = [tuple(x) for x in df.values]
for row in rows:
    session.execute(query, row)

rows=session.execute("SELECT COUNT(*) FROM sunspots")
for row in rows:
    print(row)

session.execute('DROP KEYSPACE mykeyspace') 
cluster.shutdown()

ModuleNotFoundError: No module named 'cassandra'